In [1]:
import numpy as np
import spartan.perception.utils as perception_utils
import spartan.utils.utils as utils
from spartan.perception.heightmap import HeightMap
import ros
import ros_numpy
import rosbag
import sensor_msgs

In [2]:
depth_img_filename = "/home/manuelli/spartan/data_volume/pdc/logs_proto/2018-11-16-22-22-45/processed/images/000000_depth.png"
pose_data_filename = "/home/manuelli/spartan/data_volume/pdc/logs_proto/2018-11-16-22-22-45/processed/images/pose_data.yaml"
pose_data = utils.getDictFromYamlFilename(pose_data_filename)
camera_pose = pose_data[0]
print camera_pose

{'timestamp': 1542406969249120228, 'depth_image_filename': '000000_depth.png', 'camera_to_world': {'quaternion': {'y': 0.6767476461121406, 'x': -0.7072304726782698, 'z': -0.1659785680755306, 'w': 0.11953575589147176}, 'translation': {'y': -0.008085724235044224, 'x': 0.2970949015843446, 'z': 0.8270888615743923}}, 'rgb_image_filename': '000000_rgb.png'}


In [3]:
ros_bag_filename = "/home/manuelli/spartan/data_volume/spartan/rosbag/shoe_globalwin_brown_down_right_2018-12-04-00-31-51.bag"
ros_bag = rosbag.Bag(ros_bag_filename, "r")

pointcloud_topic = "/camera_carmine_1/depth_registered/points"
pointcloud_msg = None


def copy_msg(msg):
    new_msg = sensor_msgs.msg.PointCloud2()
    new_msg.header = msg.header
    new_msg.height = msg.height
    new_msg.width = msg.width
    new_msg.fields = msg.fields
    new_msg.is_bigendian = msg.is_bigendian
    new_msg.point_step = msg.point_step
    new_msg.row_step = msg.row_step
    new_msg.data = msg.data
    new_msg.is_dense = msg.is_dense
    return new_msg

topics = [pointcloud_topic]
counter = 0
for topic, msg, t in ros_bag.read_messages(topics=topics):
#     print topic
    counter += 1
    if counter < 50:
        continue
    if topic == pointcloud_topic:
        pointcloud_msg = copy_msg(msg)
        break
        
ros_bag.close()
print pointcloud_msg.header
pc = ros_numpy.point_cloud2.pointcloud2_to_xyz_array(pointcloud_msg, remove_nans=True)


print pc.shape

# use rosrun tf tf_echo /base /camera_carmine_1_rgb_optical_frame

translation = [0.593, 0.027, 0.588]
quat_xyzw = [0.711, -0.702, 0.005, -0.041]

d = dict()
d['translation'] = dict()
d['translation']['x'] = translation[0]
d['translation']['y'] = translation[1]
d['translation']['z'] = translation[2]

d['quaternion'] = dict()
d['quaternion']['w'] = quat_xyzw[3]
d['quaternion']['x'] = quat_xyzw[0]
d['quaternion']['y'] = quat_xyzw[1]
d['quaternion']['z'] = quat_xyzw[2]


T_H_pc = utils.homogenous_transform_from_dict(d)
print T_H_pc

pc_world_frame = perception_utils.transform_pointcloud(pc, T_H_pc)


print np.max(pc_world_frame[:,2])

# print pc.shape
# print pc_world_frame.shape

# idx = int(1e5)
# print pc[idx,:]
# print pc_world_frame[idx,:]


seq: 322
stamp: 
  secs: 1543883513
  nsecs: 225326769
frame_id: "camera_carmine_1_rgb_optical_frame"
(252557, 3)
[[ 0.01437255 -0.99780307  0.064672    0.593     ]
 [-0.99862304 -0.01106066  0.05128041  0.027     ]
 [-0.05045244 -0.06531998 -0.99658811  0.588     ]
 [ 0.          0.          0.          1.        ]]
0.0886278571361


In [4]:
hm = HeightMap.make_default()
hm.insert_pointcloud_into_heightmap(pc, T_H_pc)
print "pc.shape", pc.shape
print type(hm)
print type(hm.heightmap)
print type(hm)

pc.shape (252557, 3)
<class 'spartan.perception.heightmap.HeightMap'>
<type 'numpy.ndarray'>
<class 'spartan.perception.heightmap.HeightMap'>


In [5]:
print hm._idx_valid_matrix.shape
print hm._pc_idx.shape
print "hm._pc_idx_valid.shape", hm._pc_idx_valid.shape




print hm._idx_valid.shape
print hm._idx_valid[0]
print np.count_nonzero(hm._idx_valid)
print np.count_nonzero(hm._occupancy_grid)
print np.max(hm._heightmap)

(252557, 3)
(252557, 3)
hm._pc_idx_valid.shape (19928, 3)
(252557,)
False
19928
5621
0.0975


In [23]:
heightmap = hm._heightmap
idx = np.where(hm._heightmap > 0)[0]
print idx.shape
z_height = heightmap[heightmap > 0]
xy_coords = hm._xy_grid[heightmap > 0]
print xy_coords.shape
N = xy_coords.shape[0]
pch = np.concatenate((xy_coords, z_height.reshape([N,1])), axis=1)
pch_rec = np.array(pch, dtype=[('x', float), ('y', float), ('z', float)])

(3687,)
(3687, 2)


In [28]:
pc2 = ros_numpy.point_cloud2.array_to_pointcloud2(pch_rec)
print pc2.height

3687
